In [ ]:
## Ahmad Allahham for IBM Coursera Capstone Project

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
#=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [206]:
### wikipedia request info

In [3]:
website_url = requests.get("https://es.wikipedia.org/wiki/Anexo:Localidades_de_Bogot%C3%A1").text

In [4]:
soup = BeautifulSoup(website_url,'lxml')


In [7]:
My_table = soup.find('table',{'class':'sortable wikitable'})


In [8]:
links = My_table.findAll('a')

In [9]:
Localidades = []
for link in links:
    Localidades.append(link.get('title'))
    
print(Localidades)

del Localidades[0:2]

Localidades

[None, None, 'Usaquén', 'Chapinero', 'Santa Fe (Bogotá)', 'San Cristóbal (Bogotá)', 'Usme', 'Tunjuelito', 'Bosa (Bogotá)', 'Kennedy (Bogotá)', 'Fontibón', 'Engativá', 'Suba', 'Barrios Unidos (Bogotá)', 'Teusaquillo', 'Los Mártires', 'Antonio Nariño (Bogotá)', 'Puente Aranda', 'La Candelaria', 'Rafael Uribe Uribe (Bogotá)', 'Ciudad Bolívar (Bogotá)', 'Sumapaz (Bogotá)']


['Usaquén',
 'Chapinero',
 'Santa Fe (Bogotá)',
 'San Cristóbal (Bogotá)',
 'Usme',
 'Tunjuelito',
 'Bosa (Bogotá)',
 'Kennedy (Bogotá)',
 'Fontibón',
 'Engativá',
 'Suba',
 'Barrios Unidos (Bogotá)',
 'Teusaquillo',
 'Los Mártires',
 'Antonio Nariño (Bogotá)',
 'Puente Aranda',
 'La Candelaria',
 'Rafael Uribe Uribe (Bogotá)',
 'Ciudad Bolívar (Bogotá)',
 'Sumapaz (Bogotá)']

In [10]:
df = pd.DataFrame()
df['Localidades'] = Localidades
df

,Localidades
0,Usaquén
1,Chapinero
2,Santa Fe (Bogotá)
3,San Cristóbal (Bogotá)
4,Usme
5,Tunjuelito
6,Bosa (Bogotá)
7,Kennedy (Bogotá)
8,Fontibón
9,Engativá


In [11]:
def get_coords_local(localidad, output_as='center'):
    """
    get the bounding box of a locality in WGS84 given its name

    Parameters
    ----------
    localidad : str
        name of the country in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]

    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             localidad+', Bogota, Bogota Capital District',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

In [12]:
df2 = df.copy()

latitudeCln = []
longitudeCln = []
for index, row in df2.iterrows():
    print(row[0])
    lat, long = get_coords_local(localidad=row[0], output_as='center')
    latitudeCln.append(lat)
    longitudeCln.append(long)

df2['Latitude'] = latitudeCln
df2['Longitude'] = longitudeCln

df2.shape

Usaquén
Chapinero
Santa Fe (Bogotá)
San Cristóbal (Bogotá)
Usme
Tunjuelito
Bosa (Bogotá)
Kennedy (Bogotá)
Fontibón
Engativá
Suba
Barrios Unidos (Bogotá)
Teusaquillo
Los Mártires
Antonio Nariño (Bogotá)
Puente Aranda
La Candelaria
Rafael Uribe Uribe (Bogotá)
Ciudad Bolívar (Bogotá)
Sumapaz (Bogotá)


(20, 3)

In [14]:
df2

,Localidades,Latitude,Longitude
0,Usaquén,4.694969,-74.031093
1,Chapinero,4.645377,-74.061943
2,Santa Fe (Bogotá),4.602204,-74.078837
3,San Cristóbal (Bogotá),4.548658,-74.047473
4,Usme,4.411136,-74.129108
5,Tunjuelito,4.562204,-74.127647
6,Bosa (Bogotá),4.625492,-74.200280
7,Kennedy (Bogotá),4.629682,-74.149935
8,Fontibón,4.673327,-74.144732
9,Engativá,4.708695,-74.109643


In [15]:
address = 'Bogotá, Colombia'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Bogotá are {}, {}.'.format(latitude, longitude))

The decimal coordinates of Bogotá are 4.59808, -74.0760439.


In [17]:
# make a copy of the dataframe to get it simply back if needed
df3 = df2.copy()
df3

,Localidades,Latitude,Longitude
0,Usaquén,4.694969,-74.031093
1,Chapinero,4.645377,-74.061943
2,Santa Fe (Bogotá),4.602204,-74.078837
3,San Cristóbal (Bogotá),4.548658,-74.047473
4,Usme,4.411136,-74.129108
5,Tunjuelito,4.562204,-74.127647
6,Bosa (Bogotá),4.625492,-74.200280
7,Kennedy (Bogotá),4.629682,-74.149935
8,Fontibón,4.673327,-74.144732
9,Engativá,4.708695,-74.109643


In [18]:
# create map of Bogotá using latitude and longitude values
map_bogota = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(df3['Latitude'], df3['Longitude'], df3['Localidades']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bogota)  
    
map_bogota

In [19]:
# function to repeat the exploring process to all the neighborhoods in Toronto
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Localidad', 
                  'Localidad Latitude', 
                  'Localidad Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [25]:
CLIENT_ID = 'L02I2SMFLLTF1HWNBBDNX2H00TMSHDJSBATZXHVNH0V3ONEY' # your Foursquare ID
CLIENT_SECRET = 'EQKOEYQAOCHYQVB3J52F0BOQ5HDC4LGC3PMRSKUTZWKT53RT' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L02I2SMFLLTF1HWNBBDNX2H00TMSHDJSBATZXHVNH0V3ONEY
CLIENT_SECRET:EQKOEYQAOCHYQVB3J52F0BOQ5HDC4LGC3PMRSKUTZWKT53RT


In [26]:
# Use category id 4bf58dd8d48988d16c941735 to only get the burger joints
bogota_venues_burger = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16c941735')
bogota_venues_burger.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquén,4.694969,-74.031093,La Hamburguesería,4.695310,-74.031716,Burger Joint
1,Usaquén,4.694969,-74.031093,McDonald's,4.695702,-74.031881,Burger Joint
2,Usaquén,4.694969,-74.031093,Home Burgers,4.693883,-74.032853,Burger Joint
3,Usaquén,4.694969,-74.031093,Madison,4.695443,-74.030262,Burger Joint
4,Usaquén,4.694969,-74.031093,El Taller 'Handmade Burgers & Beers',4.695879,-74.029299,Burger Joint


In [222]:
bogota_venues_burger.shape

(210, 7)

In [27]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Localidad'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [28]:
map_bogota_burger = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_burger, 'red', map_bogota_burger)
map_bogota_burger

In [29]:
bogota_venues_highschools = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d13d941735')
bogota_venues_highschools.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquén,4.694969,-74.031093,Gimnasio Los Cerros,4.694464,-74.026556,School
1,Chapinero,4.645377,-74.061943,Colegio Nuestra Señora de Chiquinquira,4.649800,-74.065643,High School
2,Chapinero,4.645377,-74.061943,Colegio del Sagrado Corazón de Jesus Bethlemitas,4.649182,-74.058036,High School
3,Chapinero,4.645377,-74.061943,Colegio Distrital Simón Rodriguez,4.653948,-74.060732,High School
4,Chapinero,4.645377,-74.061943,Colegio Ervit,4.642634,-74.066469,High School


In [30]:
bogota_venues_highschools.shape

(36, 7)

In [31]:
map_bogota_highschools = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_highschools, 'green', map_bogota_highschools)
map_bogota_highschools

In [32]:
bogota_venues_uni = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ae941735')
bogota_venues_uni.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquén,4.694969,-74.031093,Tecnologico de Monterry,4.692100,-74.034916,University
1,Usaquén,4.694969,-74.031093,Centrum Católica,4.690426,-74.038657,University
2,Usaquén,4.694969,-74.031093,UTadeo M7a,4.705390,-74.032827,University
3,Chapinero,4.645377,-74.061943,Universidad de La Salle,4.644576,-74.059379,University
4,Chapinero,4.645377,-74.061943,Paradero poli,4.637138,-74.054646,University


In [33]:
bogota_venues_uni.shape

(100, 7)

In [34]:
map_bogota_uni = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_uni, 'gold', map_bogota_uni)
map_bogota_uni

In [35]:
bogota_venues_office = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4d4b7105d754a06375d81259')
bogota_venues_office.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquén,4.694969,-74.031093,Fundacion Santa Fe de Bogota,4.695136,-74.033211,Hospital
1,Usaquén,4.694969,-74.031093,Centro Médico de la Sabana,4.696026,-74.031596,Doctor's Office
2,Usaquén,4.694969,-74.031093,Santa Ana Medical Center,4.696085,-74.032065,Doctor's Office
3,Usaquén,4.694969,-74.031093,Flormorado,4.694266,-74.032539,Business Center
4,Usaquén,4.694969,-74.031093,WeWork Usaquén,4.694304,-74.032745,Coworking Space


In [36]:
bogota_venues_office.shape

(472, 7)

In [37]:
map_bogota_office = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_office, 'fuchsia', map_bogota_office)
map_bogota_office

In [38]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Localidad').count()
    
    for n in startDf['Localidad']:
        try:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = 0

In [39]:
df_data = df3.copy()
df_data.rename(columns={'Localidades':'Localidad'}, inplace=True)
addColumn(df_data, 'Burger', bogota_venues_burger)
addColumn(df_data, 'High Schools', bogota_venues_highschools)
addColumn(df_data, 'Universities', bogota_venues_uni)
addColumn(df_data, 'Offices', bogota_venues_office)
df_data

,Localidad,Latitude,Longitude,Burger,High Schools,Universities,Offices
0,Usaquén,4.694969,-74.031093,21.0,1.0,3.0,30.0
1,Chapinero,4.645377,-74.061943,30.0,4.0,29.0,30.0
2,Santa Fe (Bogotá),4.602204,-74.078837,23.0,2.0,22.0,30.0
3,San Cristóbal (Bogotá),4.548658,-74.047473,0.0,0.0,0.0,0.0
4,Usme,4.411136,-74.129108,0.0,0.0,0.0,0.0
5,Tunjuelito,4.562204,-74.127647,3.0,2.0,0.0,28.0
6,Bosa (Bogotá),4.625492,-74.200280,0.0,1.0,0.0,22.0
7,Kennedy (Bogotá),4.629682,-74.149935,12.0,1.0,1.0,26.0
8,Fontibón,4.673327,-74.144732,3.0,3.0,2.0,28.0
9,Engativá,4.708695,-74.109643,13.0,2.0,1.0,30.0


In [40]:
# negative weight, because Jeronimo wants to open a burger joint and thus wants to avoid concurrence as much as possible
weight_burger = -1

# positive weight, because high school students are good customers
weight_schools = 1

# positive weight, because uni students are good customers
weight_uni = 1.5

# positive weight because employees are even better customers
weight_offices = 2

In [41]:
df_weighted = df_data[['Localidad']].copy()

In [42]:
df_weighted['Score'] = df_data['Burger'] * weight_burger + df_data['High Schools'] * weight_schools + df_data['Universities'] * weight_uni + df_data['Offices'] * weight_offices
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted

,Localidad,Score
16,La Candelaria,81.0
1,Chapinero,77.5
2,Santa Fe (Bogotá),72.0
13,Los Mártires,59.5
15,Puente Aranda,59.0
8,Fontibón,59.0
14,Antonio Nariño (Bogotá),57.5
11,Barrios Unidos (Bogotá),55.5
12,Teusaquillo,55.5
5,Tunjuelito,55.0


In [43]:
map_bog_result = folium.Map(location=[latitude, longitude], zoom_start=15)

bog_win = df3[df3['Localidades'] == 'La Candelaria']

for lat, lng, local in zip(bog_win['Latitude'], bog_win['Longitude'], bog_win['Localidades']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_bog_result) 

addToMap(bogota_venues_burger[bogota_venues_burger['Localidad'] == 'La Candelaria'], 'red', map_bog_result)
addToMap(bogota_venues_highschools[bogota_venues_highschools['Localidad'] == 'La Candelaria'], 'green', map_bog_result)
addToMap(bogota_venues_uni[bogota_venues_uni['Localidad'] == 'La Candelaria'], 'gold', map_bog_result)
addToMap(bogota_venues_office[bogota_venues_office['Localidad'] == 'La Candelaria'], 'fuchsia', map_bog_result)

map_bog_result

In [ ]:
### La Candelaria Locality is the best option for Jeronimo in order to open his Burger Joint